## Definitions

In [71]:
import os
import sys
import shutil
import random

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pandas_ta as ta
import quantstats as qs
qs.extend_pandas()

import numpy as np
import math

from datetime import datetime, timedelta
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline

params = {'figure.facecolor': 'w'}
plt.rcParams.update(params)

from IPython.display import display

''' Import custom Library '''
lib_path = '/workspace/202205_idx-trading/lib'
sys.path.insert(0, lib_path)
# Read Imports
from utils import read_config
from strat_utils import LQ45BaseStrategy
from data_utils import gen_combined_df, extend_price_df, handle_nan, RandomPriceData
from backtest import Backtest, HistoricalScenarioBacktest, RandomizedBacktest
sys.path.remove(lib_path)

### Parameter and Directories Definition

In [2]:
# Parameters
date_start = '2010-01-01'
date_breakpoint = '2019-01-01'
strat_class = "Momentum"
std = 1

In [3]:
# Data Directory
data_dir = '/workspace/202205_idx-trading/_data/'
lq45_dir = '/workspace/202205_idx-trading/_data/20220525_lq45/'
lq45_index_file = data_dir + '20220525_lq45_index.csv'
lq45_list = '20220525_lq45-list.txt'

## Data Preparation

### Data Loading
Note: Only In sample data is loaded

In [4]:
# Prepare Stock Tickers
with open(data_dir + lq45_list, "r") as f:
    lq45_tickers = f.read().split('\n')

## Prepare active tickers for international codes
active_tickers = [f + '.JK' for f in lq45_tickers]
active_tickers.append('LQ45')

In [5]:
# Prepare Time Series Data
nan_handle_method = 'bfill'

df_dict = {}
for ticker in tqdm(active_tickers):
    if ticker == 'LQ45':
        df_dict[ticker] = pd.read_csv(lq45_index_file)
    else:
        df_dict[ticker] = pd.read_csv(lq45_dir + ticker + '.csv')
    
    ## Take Only Date and Adjusted Close
    df_dict[ticker] = df_dict[ticker][['Date', 'Adj Close']]
    df_dict['Date'] = pd.to_datetime(df_dict[ticker]['Date'])
    df_dict[ticker].set_index(pd.DatetimeIndex(df_dict[ticker]['Date']), inplace=True)
    
    df_dict[ticker].drop('Date', axis=1, inplace=True)
    
    ## Convert Adj Close to price
    df_dict[ticker]['price'] = df_dict[ticker]['Adj Close']
    df_dict[ticker].drop('Adj Close', axis=1, inplace=True)

100%|█████████████████████████████████████████████████████| 46/46 [00:00<00:00, 62.28it/s]


In [6]:
# Generate In Sample Dataset
nan_cnt_threshold = 252*2

in_df = {}
out_df = {}
rmv_tickers = []
for ticker in tqdm(active_tickers):
    ## Take In Sample and Out Sample Data
    in_df[ticker] = df_dict[ticker][(df_dict[ticker].index >= date_start) & 
                                                (df_dict[ticker].index < date_breakpoint)]
    
    ## Check if there are too many NaN values
    if in_df[ticker]['price'].isna().sum() > nan_cnt_threshold:
        rmv_tickers.append(ticker)
        continue
    
    ## Handle NaN Values
    in_df[ticker] = handle_nan(in_df[ticker], method=nan_handle_method)
    
    ## Extend price to other values
    in_df[ticker] = extend_price_df(in_df[ticker])

# Remove tickers that only have small amounts of data
active_tickers = [t for t in active_tickers if t not in rmv_tickers]

100%|████████████████████████████████████████████████████| 46/46 [00:00<00:00, 373.61it/s]


## Strategy Design
Done by the Robert Carver method: https://www.youtube.com/watch?app=desktop&v=-aT55uRJI8Q

In [228]:
class BinaryEWMACStrategy(LQ45BaseStrategy):
    '''
    Exponential Moving Average Crossover Strategy.
    
    Signal is generated in a binary manner (buy/sell) 
    
    TODO - can be further improved by considering a continuous forecast
    '''
    def __init__(self, config=None, config_filepath=None, mode="paper_trade"):
        super().__init__(config=config, config_filepath=config_filepath, mode=mode)
        
        if config is not None:
            config_dict = config
        
        elif config_filepath is not None:
            config_dict = read_config(config_filepath)
            
        else:
            assert config is not None, "Either config or config_filepath must be available"
            
        # Strategy Parameters
        strat_params = config_dict['strat_params']
        
        self.long_only = strat_params['long_only']
        
        self.tickers = strat_params['tickers']
        self.lookback_fast = strat_params['lookback_fast']
        self.lookback_slow = strat_params['lookback_slow']
        self.vol_lookback = strat_params['vol_lookback']
    
    def prepare_indicators(self, df_dict, vol_adj=False):
        strat_df = gen_combined_df(df_dict, self.tickers, ['price'], add_pfix=True)
        
        for t in self.tickers:
            price_t = "price_" + t
            ewmac_t = "ewmac_" + t
            vol_adj_ewmac_t = "vol-adj-ewmac_" + t
            
            fast_ewma = strat_df[price_t].ewm(span=self.lookback_fast).mean()
            slow_ewma = strat_df[price_t].ewm(span=self.lookback_slow).mean()
            strat_df[ewmac_t] = fast_ewma - slow_ewma
            
            if vol_adj:
                stdev_ret = (strat_df[price_t] - strat_df[price_t].shift(1)).ewm(span=self.vol_lookback).mean()
                strat_df[vol_adj_ewmac_t] = strat_df[ewmac_t] / stdev_ret
            
        return strat_df
    
    def gen_signals(self, strat_df):
        # Signal Rules
        uptrend_signal = lambda ewmac: ewmac > 0
        downtrend_signal = lambda ewmac: ewmac <= 0
        
        # Prepare Signal
        last_signal = {}
        
        signal_tickers = ["signal_" + t for t in self.tickers]
        for signal_t in signal_tickers:
            strat_df[signal_t] = ''
            last_signal[signal_t] = ''
        
        # Generate Signals
        if not self.long_only:
            for t in signal_tickers:
                signal_t = "signal_" + t
                ewmac_t = "ewmac_" + t

                for i in range(0, len(strat_df)):
                    if i == 0:
                        strat_df[signal_t][i] = ''

                    elif last_signal[signal_t] == '':
                        if uptrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = 'long_entry'
                        elif downtrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = 'short_entry'
                        else:
                            strat_df[signal_t][i] = ''

                    elif last_signal[signal_t] == 'long_entry':
                        if uptrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = ''
                        elif downtrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = 'long_close'
                        else:
                            strat_df[signal_t][i] = ''

                    elif last_signal[signal_t] == 'short_entry':
                        if uptrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = 'short_close'
                        elif downtrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = ''
                        else:
                            strat_df[signal_t][i] = ''

                    elif last_signal[signal_t] == 'long_close' or last_signal[signal_t] == 'short_close':
                        if uptrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = 'long_entry'
                        elif downtrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = 'short_entry'
                        else:
                            strat_df[signal_t][i] = ''
                            
        elif self.long_only:
            for t in signal_tickers:
                signal_t = "signal_" + t
                ewmac_t = "ewmac_" + t

                for i in range(0, len(strat_df)):
                    if i == 0:
                        strat_df[signal_t][i] = ''
                    
                    elif last_signal[signal_t] == '':
                        if uptrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = 'long_entry'
                        elif downtrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = ''
                        else:
                            strat_df[signal_t][i] = ''

                    elif last_signal[signal_t] == 'long_entry':
                        if uptrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = ''
                        elif downtrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = 'long_close'
                        else:
                            strat_df[signal_t][i] = ''
                            
                    elif last_signal[signal_t] == 'long_close':
                        if uptrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = 'long_entry'
                        elif downtrend_signal(strat_df[ewmac_t]):
                            strat_df[signal_t][i] = ''
                        else:
                            strat_df[signal_t][i] = ''
        
        # Remove signals for tickers not within runtime
        for signal_t in signal_pair:
            strat_df.loc[strat_df.index < self.run_date_start, signal_t] = ""
        strat_df.loc[strat_df.index < self.run_date_start, 'spread_signal'] = ""
        
        return strat_df
    
    def run(self):
        self.df_data_dict = self.prepare_data()
        self.strat_df = self.prepare_indicators(self.df_data_dict)
        self.strat_df = self.gen_signals(self.strat_df)
        
        return self.strat_df

### Generate Random Data

In [218]:
r = RandomPriceData()
out_dict = {}

# Trend lengths are based on rob carver's method (gradually increase from a week up until a year)
t_lengths = [5, 10, 15, 21, 42, 64, 85, 107, 128, 150, 150, 171, 192, 213, 235, 256]

In [ ]:
random_data_tickers = []
for tl, ii in enumerate(t_lengths):
    sawtooth_data = r.random_oscillation(n_length=512, t_length=tl, amp=10, std_scale=0.2, mode='sawtooth')
    out_dict[str(ii)] = pd.DataFrame(sawtooth_data, columns=['price'], index=sawtooth_data.index)
    random_data_tickers.append(str(ii))

### Setup Strategy

In [229]:
config = {
            "run_params":{
                "base_data_dir": "/workspace/202205_idx-trading/_data/",
                "lq45_dir": "/workspace/202205_idx-trading/_data/20220525_lq45/",
                "lq45_index_filename": "20220525_lq45_index.csv",
                "lq45_list_filename": "20220525_lq45-list.txt"
            },

            "backtest_params":{
                "run_date_start": date_breakpoint,
                "run_date_end": "full"
            },

            "strat_params":{
                "long_only": True,
                "tickers": random_data_tickers,
                "lookback_fast": 16,
                "lookback_slow": 64,
                "vol_lookback": 25
            }   
}

In [230]:
s = BinaryEWMACStrategy(config=config, mode="backtest")

### Run Strategy

In [ ]:
# TODO (NOW) - Run strategy variations with different lookback periods
# generate sharpe ratio, turnover, and correlation between strategies

## Backtest

In [ ]:
# TODO - Fit strategy on real data, for portofolio adjustment

In [ ]:
# TODO - Strategy Config

In [ ]:
# TODO - WF Backtest, Historical Backtest, Randomized Backtest